# 앙상블들의 앙상블
 - 서로 다른 분류기 형태 간의 앙상블들의 앙상블
 - 동일 형식 분류기를 사용한 부트스트랩 표본을 통한 앙상블들의 앙상블

In [1]:
import pandas as pd
pd.options.display.max_columns=None

---

### 데이터 로딩

IBM에서 제공했던 HR 데이터를 활용하겠습니다.

IBM kaggle 데이터 : https://www.kaggle.com/pavansubhasht/ibm-hr-analytics-attrition-dataset

In [2]:
datasets = pd.read_csv('./inputs/HR-Employee-Attrition.csv')
datasets.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [3]:
datasets.shape

(1470, 35)

데이터를 살펴보면 categorycal features, numerical features가 함께 있습니다.

In [4]:
datasets.dtypes

Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeNumber               int64
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears   

Taget variable : *Attrition*

Yes / No -> 1 / 0으로 변경합니다
- 1 : 퇴직 Yes
- 0 : 퇴직 No

In [5]:
datasets['Attrition_idx'] = datasets['Attrition']\
    .apply(lambda x: 1 if x == 'Yes' else 0)
datasets.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_idx
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5,1
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7,0
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0,1
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2,0


### Column 전처리

In [6]:
col_names = datasets.columns
col_names

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'Attrition_idx'],
      dtype='object')

필요없는 변수들이 있다 : *EmployeeCount*, *EmployeeNumber*, *Over18*, *StandardHours*

In [7]:
print(datasets.Over18.value_counts())
print(datasets.EmployeeCount.value_counts())
print(datasets.StandardHours.value_counts())

Y    1470
Name: Over18, dtype: int64
1    1470
Name: EmployeeCount, dtype: int64
80    1470
Name: StandardHours, dtype: int64


In [8]:
# Target은 feature에서 제외한다.
col_names = col_names\
    .drop(['Attrition_idx', 'Attrition', 'Over18', 
           'EmployeeCount', 'EmployeeNumber', 'StandardHours'])

Categorical column을 다루어보자.

Catagorical column을 numerical column을 나누어보자.

In [9]:
categorical_features = []
numerical_features = []
target = 'Attrition_idx'

# feature를 2가지 형태로 구분한다.
for col in col_names:
    if datasets[col].dtype == 'O':
        categorical_features.append(col)
    else:
        numerical_features.append(col)

In [10]:
print('Categorical feature의 수 :', len(categorical_features))
print('Numerical feature의 수 :', len(numerical_features))

Categorical feature의 수 : 7
Numerical feature의 수 : 23


In [11]:
categorical_features

['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [12]:
numerical_features

['Age',
 'DailyRate',
 'DistanceFromHome',
 'Education',
 'EnvironmentSatisfaction',
 'HourlyRate',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'MonthlyIncome',
 'MonthlyRate',
 'NumCompaniesWorked',
 'PercentSalaryHike',
 'PerformanceRating',
 'RelationshipSatisfaction',
 'StockOptionLevel',
 'TotalWorkingYears',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'YearsAtCompany',
 'YearsInCurrentRole',
 'YearsSinceLastPromotion',
 'YearsWithCurrManager']

<font color=red>Categorical 데이터를 one-hot vector로 변경</font>하자. Pandas에서 `get_dummies`를 이용하자.
- Train, test set을 구분하지 않고 원핫벡터를 만드는 경우 : 해당 feature의 모든 원소들을 아는 경우, 예를 들어 회사 부서, 국가 코드의 경우에 해당한다.
- Train, test set을 구분하고 train set으로만 원핫벡터를 만드는 경우 : 해당 feature의 원소가 test set에 없는 경우가 존재할 수 있다.

In [13]:
categorical_datasets = pd.get_dummies(datasets[categorical_features])
categorical_datasets.head()

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1
1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0
2,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1
3,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
4,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0


In [14]:
numerical_datasets = datasets[numerical_features]
numerical_datasets.head()

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1102,1,2,2,94,3,2,4,5993,19479,8,11,3,1,0,8,0,1,6,4,0,5
1,49,279,8,1,3,61,2,2,2,5130,24907,1,23,4,4,1,10,3,3,10,7,1,7
2,37,1373,2,2,4,92,2,1,3,2090,2396,6,15,3,2,0,7,3,3,0,0,0,0
3,33,1392,3,4,4,56,3,1,3,2909,23159,1,11,3,3,0,8,3,3,8,7,3,0
4,27,591,2,1,1,40,3,1,2,3468,16632,9,12,3,4,1,6,3,3,2,2,2,2


Categorical dataset과 numerical dataset을 합친다. 모델의 input으로 사용할 feature이다.

In [15]:
X = pd.concat([categorical_datasets, numerical_datasets], axis=1)
X.head()

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,41,1102,1,2,2,94,3,2,4,5993,19479,8,11,3,1,0,8,0,1,6,4,0,5
1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,49,279,8,1,3,61,2,2,2,5130,24907,1,23,4,4,1,10,3,3,10,7,1,7
2,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,37,1373,2,2,4,92,2,1,3,2090,2396,6,15,3,2,0,7,3,3,0,0,0,0
3,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,33,1392,3,4,4,56,3,1,3,2909,23159,1,11,3,3,0,8,3,3,8,7,3,0
4,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,27,591,2,1,1,40,3,1,2,3468,16632,9,12,3,4,1,6,3,3,2,2,2,2


In [16]:
y = datasets[target]
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Attrition_idx, dtype: int64

### Train set과 test set을 구분

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
x_train, x_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=42)

### 모델의 성능을 평가하기 위한 metric들을 정의한 helper 함수 정의

In [19]:
from sklearn.metrics import accuracy_score, classification_report

In [20]:
# Help 함수
def get_metric(y_label, pred, set_type):
    # 1. Confusion Matrix
    print('\n {} Confusion Matrix :'.format(set_type))
    display(pd.crosstab(y_label, pred, rownames=['Actual'], colnames=['Predict']))

    # 2. Accuracy
    print('\n {} accuracy :'.format(set_type), accuracy_score(y_label, pred))

    # 3. Classification Report
    print('\n {} Classification Report : \n'.format(set_type), classification_report(y_label, pred))

---

## 서로 다른 분류기 형태 간의 앙상블들의 앙상블

- 훈련 데이터에 각 네 가지 분류기를 별도로 적용(로지스틱 회귀, 의사결정 트리, 램덤 포레스트, 에이다 부스트)
- 네 가지 분류기에 관한 확률을 계산한다. 그리고 부류 1에 관한 확률만 메타 분류기에 이용한다. 부류 0의 확률 + 부류 1의 확률 = 1이므로 하나의 확률만 계산해도 충분하다. 그렇지 않으면 다중공선성 문제가 발생한다.
- 최종 0/1 출력에 관한 4개의 확률(각 분류기에서 얻은 값) 간의 관계를 모델링하기 위해 로지스틱 회귀를 메타 분류기로 사용했다.
- 메타 분류기에 사용된 4개 변수 모두에 관한 계수를 계산하고 새로운 데이터에 적용해 관측값을 최종 부류로 분류하기 위한 계산을 한다.

### 1. 분류기에 대하여 학습을 진행

In [21]:
# Decision tree 강의에서 클래스 가중치를 구했습니다.
c_weight = {0: 0.3, 1: 0.7}

> 로지스틱 회귀 분류기

In [22]:
from sklearn.linear_model import LogisticRegression
lr_classifier = LogisticRegression(fit_intercept=True, class_weight=c_weight)

In [23]:
lr_classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight={0: 0.3, 1: 0.7}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [24]:
# 로지스틱 회귀 분류기 모델을 이용하여 예측값을 구한다.
train_pred = lr_classifier.predict(x_train)
test_pred = lr_classifier.predict(x_test)

In [25]:
# 로지스틱 회귀 분류기 모델의 성능을 구한다.
get_metric(y_train, train_pred, 'Train')
print('=' * 60)
get_metric(y_test, test_pred, 'Test')


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,779,74
1,62,114



 Train accuracy : 0.8678328474246841

 Train Classification Report : 
              precision    recall  f1-score   support

          0       0.93      0.91      0.92       853
          1       0.61      0.65      0.63       176

avg / total       0.87      0.87      0.87      1029


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,339,41
1,31,30



 Test accuracy : 0.8367346938775511

 Test Classification Report : 
              precision    recall  f1-score   support

          0       0.92      0.89      0.90       380
          1       0.42      0.49      0.45        61

avg / total       0.85      0.84      0.84       441



> 결정 트리 분류기

In [26]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight=c_weight,
    random_state=42
)

In [27]:
dt_classifier.fit(x_train, y_train)

DecisionTreeClassifier(class_weight={0: 0.3, 1: 0.7}, criterion='gini',
            max_depth=5, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [28]:
# 결정 트리 분류기 모델을 이용하여 예측값을 생성한다.
train_pred = dt_classifier.predict(x_train)
test_pred = dt_classifier.predict(x_test)

In [29]:
# 결정 트리 분류기 모델의 성능을 평가해보자.
get_metric(y_train, train_pred, 'Train')
print('=' * 60)
get_metric(y_test, test_pred, 'Test')


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,787,66
1,56,120



 Train accuracy : 0.8814382896015549

 Train Classification Report : 
              precision    recall  f1-score   support

          0       0.93      0.92      0.93       853
          1       0.65      0.68      0.66       176

avg / total       0.88      0.88      0.88      1029


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,345,35
1,36,25



 Test accuracy : 0.8390022675736961

 Test Classification Report : 
              precision    recall  f1-score   support

          0       0.91      0.91      0.91       380
          1       0.42      0.41      0.41        61

avg / total       0.84      0.84      0.84       441



> 랜덤 포레스트 분류기

In [30]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(
    n_estimators=10000,
    max_depth=6,
    min_samples_split=2,
    min_samples_leaf=1,
    class_weight=c_weight,
    random_state=42)

In [31]:
rf_classifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 0.3, 1: 0.7},
            criterion='gini', max_depth=6, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10000, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [32]:
# 랜덤 포레스트 분류기를 이용하여 예측값을 구합니다.
train_pred = rf_classifier.predict(x_train)
test_pred = rf_classifier.predict(x_test)

In [33]:
# 랜덤 포레스트 분류기의 성능을 측정합니다.
get_metric(y_train, train_pred, 'Train')
print('=' * 60)
get_metric(y_test, test_pred, 'Test')


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,846,7
1,61,115



 Train accuracy : 0.9339164237123421

 Train Classification Report : 
              precision    recall  f1-score   support

          0       0.93      0.99      0.96       853
          1       0.94      0.65      0.77       176

avg / total       0.93      0.93      0.93      1029


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,373,7
1,53,8



 Test accuracy : 0.8639455782312925

 Test Classification Report : 
              precision    recall  f1-score   support

          0       0.88      0.98      0.93       380
          1       0.53      0.13      0.21        61

avg / total       0.83      0.86      0.83       441



> 에이다 부스트 분류기

In [34]:
from sklearn.ensemble import AdaBoostClassifier
ab_classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1, class_weight=c_weight),
    n_estimators=5000,
    learning_rate=0.05,
    random_state=42
)

In [35]:
ab_classifier.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight={0: 0.3, 1: 0.7}, criterion='gini',
            max_depth=1, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.05, n_estimators=5000, random_state=42)

In [36]:
# 에이다 부스트 분류기를 이용하여 예측값을 생성합니다.
train_pred = ab_classifier.predict(x_train)
test_pred = ab_classifier.predict(x_test)

In [37]:
# 에이다 부스트 분류기의 성능을 측정합니다.
get_metric(y_train, train_pred, 'Train')
print('=' * 60)
get_metric(y_test, test_pred, 'Test')


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,809,44
1,25,151



 Train accuracy : 0.9329446064139941

 Train Classification Report : 
              precision    recall  f1-score   support

          0       0.97      0.95      0.96       853
          1       0.77      0.86      0.81       176

avg / total       0.94      0.93      0.93      1029


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,342,38
1,33,28



 Test accuracy : 0.8390022675736961

 Test Classification Report : 
              precision    recall  f1-score   support

          0       0.91      0.90      0.91       380
          1       0.42      0.46      0.44        61

avg / total       0.84      0.84      0.84       441



### 2. 분류기의 결과들을 결합

In [38]:
ensemble = pd.DataFrame()

부류 1(퇴직자)에 관한 확률을 이용하여 앙상블을 수행합니다.

> 로지스틱 회귀 분류기

In [39]:
# 로지스틱 회귀 분류기를 이용하여 부류 0, 1이 될 확률을 구합니다.
lr_results = lr_classifier.predict_proba(x_train)

lr_df = pd.DataFrame(lr_results)
lr_df.head()

,0,1
0,0.996114,0.003886
1,0.832747,0.167253
2,0.305008,0.694992
3,0.509793,0.490207
4,0.867821,0.132179


In [40]:
# 로지스틱 회귀 분류기에 대한 결과값을, 부류 1이 될 확률만 리턴받습니다.
ensemble['lr_output_one'] = lr_df[1]
ensemble.head()

,lr_output_one
0,0.003886
1,0.167253
2,0.694992
3,0.490207
4,0.132179


> 결정 트리 분류기 

In [41]:
# 결정 트리 분류기를 이용하여 부류 0, 1이 될 확률을 구합니다.
dt_results = dt_classifier.predict_proba(x_train)

dt_df = pd.DataFrame(dt_results)
dt_df.head()

,0,1
0,0.903002,0.096998
1,0.903002,0.096998
2,0.072289,0.927711
3,1.000000,0.000000
4,0.904110,0.095890


In [42]:
# 결정 트리 분류기에 대한 결과값을, 부류 1이 될 확률만 리턴받습니다.
ensemble['dt_output_one'] = dt_df[1]
ensemble.head()

,lr_output_one,dt_output_one
0,0.003886,0.096998
1,0.167253,0.096998
2,0.694992,0.927711
3,0.490207,0.000000
4,0.132179,0.095890


> 랜덤 포레스트 분류기

In [43]:
# 랜덤 포레스트 분류기를 이용하여 부류 0, 1이 될 확률을 구합니다.
rf_results = rf_classifier.predict_proba(x_train)

rf_df = pd.DataFrame(rf_results)
rf_df.head()

,0,1
0,0.929821,0.070179
1,0.876245,0.123755
2,0.293263,0.706737
3,0.892285,0.107715
4,0.866287,0.133713


In [44]:
# 랜덤 포레스트 분류기에 대한 결과값을, 부류 1이 될 확률만 리턴받습니다.
ensemble['rf_output_one'] = rf_df[1]
ensemble.head()

,lr_output_one,dt_output_one,rf_output_one
0,0.003886,0.096998,0.070179
1,0.167253,0.096998,0.123755
2,0.694992,0.927711,0.706737
3,0.490207,0.000000,0.107715
4,0.132179,0.095890,0.133713


> 에이다 부스트 분류기

In [45]:
# 에이다 부스트 분류기를 이용하여 부류 0, 1이 될 확률을 구합니다.
ab_results = ab_classifier.predict_proba(x_train)

ab_df = pd.DataFrame(ab_results)
ab_df.head()

,0,1
0,0.510826,0.489174
1,0.506360,0.493640
2,0.498725,0.501275
3,0.504056,0.495944
4,0.501423,0.498577


In [46]:
# 에이다 부스트 분류기에 대한 결과값을, 부류 1이 될 확률만 리턴받습니다.
ensemble['ab_output_one'] = ab_df[1]
ensemble.head()

,lr_output_one,dt_output_one,rf_output_one,ab_output_one
0,0.003886,0.096998,0.070179,0.489174
1,0.167253,0.096998,0.123755,0.493640
2,0.694992,0.927711,0.706737,0.501275
3,0.490207,0.000000,0.107715,0.495944
4,0.132179,0.095890,0.133713,0.498577


In [47]:
ensemble = pd.concat([ensemble, pd.DataFrame(y_train).reset_index(drop=True)], axis=1)

In [48]:
ensemble.head()

,lr_output_one,dt_output_one,rf_output_one,ab_output_one,Attrition_idx
0,0.003886,0.096998,0.070179,0.489174,0
1,0.167253,0.096998,0.123755,0.493640,0
2,0.694992,0.927711,0.706737,0.501275,1
3,0.490207,0.000000,0.107715,0.495944,0
4,0.132179,0.095890,0.133713,0.498577,0


### 3. 메타 분류기 학습

In [49]:
meta_classifier = LogisticRegression(fit_intercept=False)

In [50]:
meta_classifier.fit(
    ensemble[['lr_output_one', 'dt_output_one', 'rf_output_one', 'ab_output_one']],
    ensemble['Attrition_idx'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### 4. 테스트 셋에 적용 가능하도록 테스트 셋에 대한 예측값 구하기

In [51]:
ensemble_test = pd.DataFrame()

In [52]:
# 로지스틱회귀 분류기에 대한 결과값을, 부류 1이 될 확률만 리턴받습니다.
ensemble_test['lr_output_one'] = pd.DataFrame(lr_classifier.predict_proba(x_test))[1]
ensemble_test['dt_output_one'] = pd.DataFrame(dt_classifier.predict_proba(x_test))[1]
ensemble_test['rf_output_one'] = pd.DataFrame(rf_classifier.predict_proba(x_test))[1]
ensemble_test['ab_output_one'] = pd.DataFrame(ab_classifier.predict_proba(x_test))[1]
ensemble_test.head()

,lr_output_one,dt_output_one,rf_output_one,ab_output_one
0,0.143752,0.218750,0.229717,0.498461
1,0.031456,0.096998,0.111377,0.494309
2,0.483724,0.564516,0.484103,0.502946
3,0.011683,0.096998,0.111708,0.496098
4,0.120813,0.095890,0.137883,0.495663


### 5. 메타분류기 성능 측정

In [53]:
# 학습 셋에 대한 예측값
train_pred = meta_classifier.predict(
    ensemble[['lr_output_one', 'dt_output_one', 'rf_output_one', 'ab_output_one']],)

In [54]:
# 테스트 셋에 대한 예측값
test_pred = meta_classifier.predict(
    ensemble_test[['lr_output_one', 'dt_output_one', 'rf_output_one', 'ab_output_one']],)

In [55]:
get_metric(y_train, train_pred, 'Train')
print('=' * 60)
get_metric(y_test, test_pred, 'Test')


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,840,13
1,53,123



 Train accuracy : 0.9358600583090378

 Train Classification Report : 
              precision    recall  f1-score   support

          0       0.94      0.98      0.96       853
          1       0.90      0.70      0.79       176

avg / total       0.93      0.94      0.93      1029


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,367,13
1,42,19



 Test accuracy : 0.8752834467120182

 Test Classification Report : 
              precision    recall  f1-score   support

          0       0.90      0.97      0.93       380
          1       0.59      0.31      0.41        61

avg / total       0.86      0.88      0.86       441



### 6. 개별 분류기에 대한 평가

각각의 분류기에 대해서 평가 : 로지스틱 회귀 분류기를 메타 분류기로 사용했으면 각 계수(coefficients)를 살펴본다.

In [56]:
meta_classifier.coef_

array([[ 0.87580106,  1.11820544,  6.58717132, -8.23960522]])

에이다부스트의 경우 성능을 끌어내리고 있는것처럼 보인다.

따라서, 에이다부스트의 파라미터를 조절해보고 성능을 살펴보거나, 에이다부스트를 제거하고 성능을 살펴본다.

## 동일 형식 분류기를 사용한 부트스트랩 표본을 통한 앙상블들의 앙상블

- 훈련 데이터에서 부트스트랩 표본을 추출한다.
- 각 표본에 관해 매번 에이다 부스트 모델을 학습한다.
- 개별 에이다 부스트 모델의 결과는 bagging 분류기를 통해 합쳐진다.

분산의 축소가 여전히 성능을 향상시키는 매우 유연한 모델에 관해 적절하다(의사 결정 트리, 랜덤 포레스트 등).

### 1. 기본 분류기 및 부트스트랩 샘플을 학습시킬 모델 생성

In [57]:
# 에이다 부스트 모델에 사용할 기본 분류기(의사결정 그루터기)
base_learner = DecisionTreeClassifier(max_depth=1, class_weight=c_weight)

In [58]:
# 부트스트랩 샘플을 학습시킬 에이다 부스트 모델
ab_classifier = AdaBoostClassifier(base_estimator=base_learner,
                                   n_estimators=500,
                                   learning_rate=0.05,
                                   random_state=42)

In [59]:
ab_classifier.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight={0: 0.3, 1: 0.7}, criterion='gini',
            max_depth=1, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=0.05, n_estimators=500, random_state=42)

에이다 부스트의 기본성능을 살펴보자.

In [60]:
train_pred = ab_classifier.predict(x_train)
test_pred = ab_classifier.predict(x_test)

In [61]:
get_metric(y_train, train_pred, 'Train')
print('=' * 60)
get_metric(y_test, test_pred, 'Test')


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,806,47
1,63,113



 Train accuracy : 0.8931000971817298

 Train Classification Report : 
              precision    recall  f1-score   support

          0       0.93      0.94      0.94       853
          1       0.71      0.64      0.67       176

avg / total       0.89      0.89      0.89      1029


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,349,31
1,34,27



 Test accuracy : 0.8526077097505669

 Test Classification Report : 
              precision    recall  f1-score   support

          0       0.91      0.92      0.91       380
          1       0.47      0.44      0.45        61

avg / total       0.85      0.85      0.85       441



### 2. Bootstrapping samples를 생성하고 각 샘플에 분류기를 학습

`BaggingClassifier`는 내부에 bootstrap 기능이 있고, 학습시킬 분류기를 1개 지정할 수 있다. 따라서 이것을 사용하면 된다.

In [62]:
from sklearn.ensemble import BaggingClassifier

bag_classifier = BaggingClassifier(
    base_estimator=ab_classifier,
    n_estimators = 50,
    max_samples=1.0,
    max_features=1.0,
    bootstrap=True,
    bootstrap_features=False,
    n_jobs=-1,
    random_state=42)

In [63]:
bag_classifier.fit(x_train, y_train)

BaggingClassifier(base_estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight={0: 0.3, 1: 0.7}, criterion='gini',
            max_depth=1, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_sam...None,
            splitter='best'),
          learning_rate=0.05, n_estimators=500, random_state=42),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=-1, oob_score=False,
         random_state=42, verbose=0, warm_start=False)

### 3. 성능 측정

In [64]:
train_pred = bag_classifier.predict(x_train)
test_pred = bag_classifier.predict(x_test)

In [65]:
get_metric(y_train, train_pred, 'Train')
print('=' * 60)
get_metric(y_test, test_pred, 'Test')


 Train Confusion Matrix :


Predict,0,1
Actual,,
0,824,29
1,69,107



 Train accuracy : 0.9047619047619048

 Train Classification Report : 
              precision    recall  f1-score   support

          0       0.92      0.97      0.94       853
          1       0.79      0.61      0.69       176

avg / total       0.90      0.90      0.90      1029


 Test Confusion Matrix :


Predict,0,1
Actual,,
0,359,21
1,36,25



 Test accuracy : 0.8707482993197279

 Test Classification Report : 
              precision    recall  f1-score   support

          0       0.91      0.94      0.93       380
          1       0.54      0.41      0.47        61

avg / total       0.86      0.87      0.86       441

